# SNAP IdePix

IdePix will be used to produce a land/water mask for S2 images. The `subprocess` package allows SNAP to be accessed via CLI.

## Setup

In [95]:
import os
import numpy as np
import pandas as pd

In [96]:
import matplotlib.pyplot as plt

In [97]:
import json

In [98]:
import geopandas as gpd

In [131]:
from shapely.geometry import box

In [100]:
from pathlib import Path

In [101]:
import subprocess

In [102]:
DATA_DIR = '../data/'
FIGURES_DIR = '../saved_figures/'
GRAPHS_DIR = '../graphs/'

In [161]:
with open(os.path.join(GRAPHS_DIR, 's2_subset.xml'), 'r') as f:
    subset_template = f.read()

In [158]:
with open(os.path.join(GRAPHS_DIR, 'idepix_landwater.xml'), 'r') as f:
    idepix_template = f.read()

In [159]:
with open(os.path.join(DATA_DIR, 'tile_buffer_wkt.json'), 'r') as f:
    tiles_wkt_dict = json.load(f)

## Functions to interact with SNAP

In [153]:
def run_snap_gpt(graph_path, output_log="gpt_output.log", memory="4G"):
    gpt_executable = str(Path.home() / "esa-snap" / "bin" / "gpt")

    cmd = [gpt_executable, graph_path, "-c", memory]

    try:
        with open(output_log, 'w') as log_file:
            process = subprocess.run(cmd, stdout=log_file, stderr=subprocess.STDOUT, check=True)
        print(f"SNAP processing complete. Output logged to {output_log}")
    except subprocess.CalledProcessError as e:
        print(f"Error: SNAP GPT failed. See log: {output_log}")
        raise

In [154]:
def create_graph_file(template, output_path="graph.xml", **kwargs):
    content = template.format(**kwargs)
    with open(output_path, "w") as f:
        f.write(content)
    return output_path

In [155]:
def total_bounds_to_wkt(gdf):
    minx, miny, maxx, maxy = gdf.total_bounds
    bbox_geom = box(minx, miny, maxx, maxy)
    return bbox_geom.wkt

## Processing workflow - broken

In [140]:
download_dir = os.path.join(DATA_DIR, "s2_downloads")

In [141]:
results_dir = os.path.join(DATA_DIR, "s2_processed")

In [142]:
dir_content = os.listdir(download_dir)

In [162]:
for fn in dir_content:
    fp = os.path.join(download_dir, fn)  # fn is a tileId

    if os.path.isdir(fp):
        # Processing results directory
        results_fp = os.path.join(results_dir, fn)
        os.makedirs(results_fp, exist_ok=True)
        
        safe_dirs = [x for x in os.listdir(fp) if x.endswith('.SAFE')]
        if len(safe_dirs) != 1:  # explicit check, should be only one SAFE file
            raise ValueError(f"Expected exactly one .SAFE file in {fp}, found {len(safe_dirs)}")
        safe_fn = safe_dirs[0]
        safe_fp = os.path.join(fp, safe_fn)

        # Get subset region
        gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkt(tiles_wkt_dict[fn]), crs="EPSG:4326")
        subset_region = total_bounds_to_wkt(gdf)
        
        try:
            # Run subset process
            print(f"[{fn}] Running subset graph...")
            
            subset_output_fp = os.path.join(results_fp, f'subset-{fn}.nc')
            graph_path = create_graph_file(subset_template,
                                           input_file=safe_fp,
                                           output_file=subset_output_fp,
                                           subset_region=subset_region)
            output_log = os.path.join(results_fp, f'log-subset-{fn}.log')
            run_snap_gpt(graph_path, output_log)
            
            # # Run IdePix process
            # print(f"[{fn}] Running Idepix...")
            
            # idepix_output_fp = os.path.join(results_fp, f'idepix-{fn}.nc')
            # graph_path = create_graph_file(idepix_template,
            #                                input_file=subset_output_fp,
            #                                output_file=idepix_output_fp)
            # output_log = os.path.join(results_fp, f'log-idepix-{fn}.log')
            # run_snap_gpt(graph_path, output_log)

        except Exception as e:
            print(f"[{fn}] Error processing: {e}")

[30VWL] Running subset graph...
Error: SNAP GPT failed. See log: ../data/s2_processed/30VWL/log-subset-30VWL.log
[30VWL] Error processing: Command '['/home/chanon/esa-snap/bin/gpt', 'graph.xml', '-c', '4G']' returned non-zero exit status 1.
[32UNG] Running subset graph...


KeyboardInterrupt: 